In [19]:
%pip install modules

Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install attention

Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip install seq2seq

Note: you may need to restart the kernel to use updated packages.


In [29]:
from attention.attention import Attention

In [35]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [34]:
%pip install bs4



   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -------- ------------------------------- 30.7/147.9 kB ? eta -:--:--
   -------- ------------------------------- 30.7/147.9 kB ? eta -:--:--
   ---------------------------- --------- 112.6/147.9 kB 939.4 kB/s eta 0:00:01
   ---------------------------- --------- 112.6/147.9 kB 939.4 kB/s eta 0:00:01
   -------------------------------------- 147.9/147.9 kB 678.3 kB/s eta 0:00:00


In [42]:
data=pd.read_csv("C:/Users/panch/AppData/Local/Programs/Python/Python311/Lib/site-packages/astunparse-1.6.3.dist-info/Reviews.csv",nrows=100000)

data.drop_duplicates(subset=['Text'],inplace=True)  #dropping duplicates
data.dropna(axis=0,inplace=True)   #dropping na

In [43]:
data['Text'][:10]

0    I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labr...
1             Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
2    This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with ...
3    If you are looking for the secret ingredient in Robitussin I believe I have found it.  I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda.  The fl...
4                                                               Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taff

In [65]:
%pip install contractions
import contractions

Note: you may need to restart the kernel to use updated packages.


In [49]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\panch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
stop_words = set(stopwords.words('english')) 
def text_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contractions.fix(t) if t in contractions.contractions_dict else t for t in newString.split(" ")])   
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t))

In [51]:
data['Summary'][:10]

0                            Good Quality Dog Food
1                                Not as Advertised
2                            "Delight" says it all
3                                   Cough Medicine
4                                      Great taffy
5                                       Nice Taffy
6    Great!  Just as good as the expensive brands!
7                           Wonderful, tasty taffy
8                                       Yay Barley
9                                 Healthy Dog Food
Name: Summary, dtype: object

In [52]:
def summary_cleaner(text):
    newString = re.sub('"','', text)
    newString = ' '.join([contractions.fix(t) if t in contractions.contractions_dict else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:                                 
            newString=newString+i+' '  
    return newString

#Call the above function
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(summary_cleaner(t))

data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary
data['cleaned_summary'].replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [53]:
data['cleaned_summary'] = data['cleaned_summary'].apply(lambda x : '_START_ '+ x + ' _END_')

In [54]:
for i in range(5):
    print("Review:",data['cleaned_text'][i])
    print("Summary:",data['cleaned_summary'][i])
    print("\n")

Review: bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better
Summary: _START_ good quality dog food  _END_


Review: product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo
Summary: _START_ not as advertised  _END_


Review: confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story lewis lion witch wardrobe treat seduces edmund selling brother sisters witch
Summary: _START_ delight says it all  _END_


Review: looking secret ingredient robitussin believe found got addition root beer extract ordered made cherry soda flavor medicinal
Summary: _START_ cough medicine  _END_


Review: great taffy great price wide assortment yummy taffy delivery quick taffy lo

In [68]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed

# Assuming you have your dataset loaded into 'data' DataFrame
# Preprocessing the data
max_text_len = 100
max_summary_len = 10

data = pd.read_csv("C:/Users/panch/AppData/Local/Programs/Python/Python311/Lib/site-packages/astunparse-1.6.3.dist-info/Reviews.csv")

# Drop duplicates and NA values
data.drop_duplicates(subset=['Text'], inplace=True)
data.dropna(axis=0, inplace=True)

# Tokenization
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(data['Text']))
x_vocab_size = len(x_tokenizer.word_index) + 1

# Padding
x_seq = x_tokenizer.texts_to_sequences(data['Text'])
x_padded = pad_sequences(x_seq, maxlen=max_text_len, padding='post')

# Tokenization for summaries
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(data['Summary']))
y_vocab_size = len(y_tokenizer.word_index) + 1

# Padding for summaries
y_seq = y_tokenizer.texts_to_sequences(data['Summary'])
y_padded = pad_sequences(y_seq, maxlen=max_summary_len, padding='post')

# Define the model
latent_dim = 300

# Encoder
encoder_inputs = Input(shape=(max_text_len,))
enc_emb = Embedding(x_vocab_size, latent_dim, trainable=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(y_vocab_size, latent_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(y_vocab_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model
model.fit(
    [x_padded, y_padded[:, :-1]],
    np.expand_dims(y_padded[:, 1:], -1),
    epochs=5,
    batch_size=64,
    validation_split=0.2
)

# Inference Model
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    dec_emb,
    initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_target_word_index = dict((i, word) for word, i in y_tokenizer.word_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = y_tokenizer.word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if sampled_token != 'end':
            decoded_sentence += ' ' + sampled_token

        # Exit condition: either hit max length or find stop word.
        if sampled_token == 'end' or len(decoded_sentence.split()) >= (max_summary_len-1):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.strip()

# Testing the model
for i in range(5):
    print("Review:", data['Text'][i])
    print("Original summary:", data['Summary'][i])
    print("Predicted summary:", decode_sequence(x_padded[i].reshape(1, max_text_len)))
    print()


Epoch 1/5
4920/4920 [==============================] - 22160s 5s/step - loss: 2.1392 - val_loss: 1.8673
Epoch 2/5
4920/4920 [==============================] - 37084s 8s/step - loss: 1.7303 - val_loss: 1.7181
Epoch 3/5
4920/4920 [==============================] - 13512s 3s/step - loss: 1.5488 - val_loss: 1.6550
Epoch 4/5
4920/4920 [==============================] - 15688s 3s/step - loss: 1.4192 - val_loss: 1.6295
Epoch 5/5
4920/4920 [==============================] - 48625s 10s/step - loss: 1.3123 - val_loss: 1.6291
Review: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
Original summary: Good Quality Dog Food
1/1 [==============================] - 0s 44ms/step


KeyError: 0

In [69]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_target_word_index = dict((i, word) for word, i in y_tokenizer.word_index.items())
reverse_target_word_index[0] = '<unk>'  # Add missing token index 0

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))

    # Choose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = y_tokenizer.word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index.get(sampled_token_index, '<unk>')  # Use <unk> for unknown tokens

        if sampled_token != 'end':
            decoded_sentence += ' ' + sampled_token

        # Exit condition: either hit max length or find stop word.
        if sampled_token == 'end' or len(decoded_sentence.split()) >= (max_summary_len-1):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.strip()
# Reverse-lookup token index to decode sequences back to something readable.
reverse_target_word_index = dict((i, word) for word, i in y_tokenizer.word_index.items())
reverse_target_word_index[0] = '<unk>'  # Add missing token index 0

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))

    # Choose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = y_tokenizer.word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index.get(sampled_token_index, '<unk>')  # Use <unk> for unknown tokens

        if sampled_token != 'end':
            decoded_sentence += ' ' + sampled_token

        # Exit condition: either hit max length or find stop word.
        if sampled_token == 'end' or len(decoded_sentence.split()) >= (max_summary_len-1):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.strip()


In [71]:
for i in range(5):
    print("Review:", data['Text'][i])
    print("Original summary:", data['Summary'][i])
    print("Predicted summary:", decode_sequence(x_padded[i].reshape(1, max_text_len)))
    print()

Review: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
Original summary: Good Quality Dog Food
1/1 [==============================] - 0s 49ms/step
Predicted summary: your dog <unk> <unk> <unk> <unk> <unk> <unk> <unk>

Review: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
Original summary: Not as Advertised
1/1 [==============================] - 0s 40ms/step
Predicted summary: your day <unk> <unk> <unk> <unk> <unk> <unk> <unk>

Review: This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with pow

In [81]:
for i in range(5):
    print("Review:", data['Text'][i])
    print("Summary:", data['Summary'][i])
    
    print()

Review: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
Summary: Good Quality Dog Food

Review: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
Summary: Not as Advertised

Review: This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into s